In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
#read
unsw = pd.read_csv("../datasets/UNSW_NB15_training-set.csv")

In [3]:
# Load data
"""Load unsw dataset"""
y = unsw.iloc[:,-1:]
#one-hot-encode parameters
proto = pd.get_dummies(unsw['proto'])
state = pd.get_dummies(unsw['state'])
service = pd.get_dummies(unsw['service'])
service = service.iloc[:,1:]
raw_data = unsw.values

#remove encoded parameters and add one hot
for x in ['proto', 'state', 'service','label','attack_cat']:
    unsw = unsw.drop(x,axis = 1)
    
for x in [proto, state, service]:
    unsw = unsw.join(x)
    
X = unsw.iloc[:,:-2]

### First Node

In [4]:
X1 = X[:30000] 
y1 = y[:30000]

### Second learner

In [5]:
X2 = X[30000:]
y2 = y[30000:]

In [6]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.33, random_state=42)
clf1 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', 
                   random_state=13, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False).fit(x_train1,y_train1)
y_hat = clf1.predict(x_test1)
accuracy_score(y_test1,y_hat)


/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9138383838383838

In [7]:
coef1 = clf1.coef_
inter1 = clf1.intercept_

In [9]:
type(inter1)

numpy.ndarray

In [38]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.33, random_state=42)
clf2 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', 
                   random_state=13, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False).fit(x_train2,y_train2)
y_hat = clf1.predict(x_test2)
accuracy_score(y_test2,y_hat)

/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5821656050955414

In [39]:
coef2 = clf2.coef_
inter2 = clf2.intercept_

In [40]:
coeffl = (coef1 + coef2) / 2
interfl = (inter1 + inter2) / 2

In [41]:
clf1.intercept_ = interfl
clf1.coef_ = coeffl
y_hat = clf1.predict(x_test1)
accuracy_score(y_test1,y_hat)

0.9204040404040404

In [42]:
clf2.intercept_ = interfl
clf2.coef_ = coeffl
y_hat = clf1.predict(x_test2)
accuracy_score(y_test2,y_hat)

0.5827446438911407